In [1]:
from selenium import webdriver
from time import sleep
import cdconfig
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#cdconfig.CHROMELOC
driver=webdriver.Chrome(executable_path=cdconfig.CHROMELOC)

C:\Users\elenc\AppData\Local\Temp/ipykernel_19760/831544219.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(executable_path=cdconfig.CHROMELOC)


In [3]:
# head over to garage and moving sales
driver.get("https://phoenix.craigslist.org/search/gms")

In [4]:
# switch to gridview to make grabbing many hrefs easier
try_click = driver.find_element_by_id("gridview").click()
sleep(1)
try_click = driver.find_element_by_id("picview").click()
sleep(1)

C:\Users\elenc\AppData\Local\Temp/ipykernel_19760/1026149032.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  try_click = driver.find_element_by_id("gridview").click()
C:\Users\elenc\AppData\Local\Temp/ipykernel_19760/1026149032.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  try_click = driver.find_element_by_id("picview").click()


In [5]:
# gather up distinct listing urls to crawl
listing_urls = []
bs = BeautifulSoup(driver.page_source, 'html.parser')
block = bs.find(id="search-results")
for listing in block.find_all('a'):
    lurl= listing['href']
    if (lurl != '#') & (lurl not in listing_urls):
        listing_urls.append(lurl)

In [7]:
# decided to add TV and table for fun after seeing content
# I thought about appending "person, woman, man, camera, tv" but that seemed silly.  After all, I am not a president.
burgled_words = ["mattress", "cabinet", "wrench", "tv", "table"]

evidence = {"URL":[], "CLUES":[]}

def crawl_listing(url):
    driver.get(url)
    lsoup = BeautifulSoup(driver.page_source, 'html.parser')
    sleep(1)
    lblock = lsoup.find(id="postingbody")
    #drop the text to lower case and look for our clues
    recon = lblock.text.lower()
    clues=[]
    # grab all the clues
    for clue in burgled_words:
        if clue in recon:
            clues.append(clue)
    # if we found one or more, store the URL and all of the clues
    if len(clues)>0:
        evidence["URL"].append(url)
        evidence["CLUES"].append(" ".join(clues))

# iterate through all of the URLs / inspect
for lurl in listing_urls:
    crawl_listing(lurl)
        
# save our evidence
df = pd.DataFrame.from_dict(evidence)
df.to_csv('forensics_report.txt')
df

,URL,CLUES
0,https://phoenix.craigslist.org/nph/gms/d/phoen...,tv table
1,https://phoenix.craigslist.org/nph/gms/d/phoen...,tv
2,https://phoenix.craigslist.org/nph/gms/d/phoen...,tv table
3,https://phoenix.craigslist.org/evl/gms/d/mesa-...,table
4,https://phoenix.craigslist.org/evl/gms/d/mesa-...,mattress tv table
5,https://phoenix.craigslist.org/evl/gms/d/sun-c...,table
6,https://phoenix.craigslist.org/evl/gms/d/scott...,cabinet tv table
7,https://phoenix.craigslist.org/nph/gms/d/scott...,table
8,https://phoenix.craigslist.org/cph/gms/d/phoen...,table
9,https://phoenix.craigslist.org/wvl/gms/d/palo-...,table
